In [1]:
from lxml import etree
import re
import sys

In [2]:
def owl2medlee_for_hpo(owl,medlee=None):
    
    if not medlee:
        medlee = re.sub('(^.+?\.)owl$','\g<1>medlee.txt',owl)

    tree = etree.parse(owl)
    root = tree.getroot()
    id = ''
    label = ''
    synonym = ''
    i = 0
    with open(medlee,'w+') as o:
        for ele1 in root.findall('owl:Class', root.nsmap):
            for ele2 in ele1.findall('oboInOwl:id',root.nsmap):
                m = re.match('^(HP:\d+)$', ele2.text)
                if m:
                    id = ele2.text
                    i += 1
                    for ele3 in ele1.findall('rdfs:label', root.nsmap):
                        label = ele3.text
                        o.write(id + '\t' + label + '\t' + label + '\n')
                    for ele4 in ele1.findall('efo:alternative_term',root.nsmap):
                        synonym = ele4.text
                        o.write(id + '\t' + label + '\t' + synonym + '\n')
                else:
                    next
    print('{0} Ids processed!'.format(i))
    return None

In [12]:
def owl2medlee_for_ordo(owl,medlee=None):
    medlee = re.sub('(^.+?\.)owl$','\g<1>medlee.txt',owl)
    tree = etree.parse(owl)
    root = tree.getroot()
    id = ''
    label = ''
    synonym = ''
    i = 0
    with open(medlee,'w+') as o:
        for ele1 in root.findall('owl:Class', root.nsmap):
            for ele2 in ele1.findall('skos:notation',root.nsmap):
                m = re.match('^(ORPHA:\d+)$',ele2.text)
                if m :
                    id = ele2.text
                    i +=1
                    for ele3 in ele1.findall('rdfs:label',root.nsmap):
                        label = ele3.text
                        o.write(id + '\t' + label + '\t' + label + '\n')
                    for ele4 in ele1.findall('efo:alternative_term',root.nsmap):
                        synonym = ele4.text
                        o.write(id + '\t' + label + '\t' + synonym + '\n')
                else:
                    next
    print('{0} Ids processed!'.format(i))
    return None          

In [34]:
def trim_included(string):
    string = re.sub(', INCLUDED$','',string)
    return(string)

In [43]:
def mimTitles2medlee(mimTitle,medlee=None):
    medlee = re.sub('(^.+?\.)txt$','\g<1>medlee.txt',mimTitle)
    id = ''
    label = ''
    synonym = ''
    i = 0
    with open(medlee,'w+') as o:
        omim = open(mimTitles,'r+')
        '''
        http://omim.org/help/faq#1_3
        '''
        for line in omim.readlines():
            if not re.match('^#.+?',line):
                cols = line.strip('\n').split('\t')
                prefix = cols[0]
                if prefix in ['Number Sign','Percent','NULL']:
                    id = cols[1]
                    i += 1
                    labels = cols[2].split(';')

                    label = trim_included(labels[0])

                    o.write(id + '\t' + label + '\t' + label + '\n')
                    if len(labels) > 1:
                        label_symbol = trim_included(labels[1])
                        o.write(id + '\t' + label + '\t' + label_symbol + '\n')

                    synonym1 = cols[3]
                    synonym2 = cols[4]
                    for syn in [synonym1,synonym2]:
                        inds = syn.split(';;')
                        for ind in inds:
                            if ind:

                                ts = ind.split(';')
                                title = trim_included(ts[0])
                                o.write(id + '\t' + label + '\t' + title + '\n')
                                if len(ts) > 1:
                                    symbol = trim_included(ts[1])
                                    o.write(id + '\t' + label + '\t' + symbol + '\n')
        omim.close()
    print('{0} Ids processed!'.format(i))
    return None          

In [13]:
if __name__ == '__main__':
    owl2medlee_for_hpo(sys.argv[0])
    owl2medlee_for_ordo(sys.argv[1])
    mimTitles2medlee(sys.argv[2])

9079 Ids processed!


In [44]:
mimTitles2medlee('/Users/cl3720/Projects/CrownNotes/src/main/python/mimTitles.txt')

8666 Ids processed!


True